In [1]:
import pandas as pd
import yaml
from functions import trim_and_lower, convert_to_upper
import re
from collections import Counter

In [2]:
try:
    with open("../config.yaml", "r") as file:
        config = yaml.safe_load(file)
except:
    print("The configuration file was not found!")

In [3]:
path = config['data']['raw']['raw_1']
df = pd.read_excel(path)
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
# Remove whitespace, change strings to lowercase and insert "_" in place of " " in column names
df = trim_and_lower(df)

# Drop all rows where "customer_id" is unknown
df = df.dropna(subset=['customer_id']).reset_index()

# Get all indices greater than or equal to 50000 and drop these indices from the DataFrame
indices_to_drop = df.index[df.index >= 50000]
df = df.drop(index=indices_to_drop)

# Cast floats to int and then string in "customer_id" column
df.customer_id = df.customer_id.astype(int).astype(str)

# Filter out all rows where the quantity is equal to or below 0 (assuming these are returns)
df = df[df.quantity >= 1]

# Rename some columns
col_names = {"stockcode": "product_id", "invoicedate": "invoice_date", "invoice": "invoice_number", "price": "unit_price"}
df = df.rename(columns=col_names)

# Change "invoice_date" frome datetime to date format
df.invoice_date = df.invoice_date.dt.date

# Create new column called "line_price", which is "quantity" * "unit_price"
df["line_price"] = df.quantity * df.unit_price

# Convert "product_id" back to uppercase
df['product_id'] = df['product_id'].apply(convert_to_upper)

# Replace "c/cover" with "cushion cover" for consistency in the description column
# Function to replace words in a text
replacements = {
    "c/cover": "cushion cover",
    "hot water bottle": "hot-water-bottle"
}

def replace_words(text, replacements):
    for old, new in replacements.items():
        text = text.replace(old, new)
    return text

# Apply the replacements to the 'description' column
df['description'] = df['description'].apply(lambda x: replace_words(x, replacements))


df

,index,invoice_number,product_id,description,quantity,invoice_date,unit_price,customer_id,country,line_price
0,0,489434,85048,15cm christmas glass ball 20 lights,12,2009-12-01,6.95,13085,united kingdom,83.40
1,1,489434,79323P,pink cherry lights,12,2009-12-01,6.75,13085,united kingdom,81.00
2,2,489434,79323W,white cherry lights,12,2009-12-01,6.75,13085,united kingdom,81.00
3,3,489434,22041,"record frame 7"" single size",48,2009-12-01,2.10,13085,united kingdom,100.80
4,4,489434,21232,strawberry ceramic trinket box,24,2009-12-01,1.25,13085,united kingdom,30.00
...,...,...,...,...,...,...,...,...,...,...
49995,71548,495875,21201,tropical honeycomb paper garland,3,2010-01-27,2.55,16550,united kingdom,7.65
49996,71549,495875,44234,assorted circular mobile,5,2010-01-27,0.21,16550,united kingdom,1.05
49997,71550,495875,47590B,pink happy birthday bunting,2,2010-01-27,5.45,16550,united kingdom,10.90
49998,71551,495875,21745,gaolers keys decorative garden,12,2010-01-27,3.75,16550,united kingdom,45.00


In [137]:
# Function to hyphenate specified word combinations
def hyphenate_words(text):
    # Define word combinations to hyphenate
    word_combinations = [
        ('gift', 'tape'),
        ('gift', 'wrap'),
        ('photo', 'frame'),
        ('cushion', 'cover'),
        ('passport', 'cover'),
        ('food', 'cover'),
        ('ironing', 'board'),
        ('egg', 'basket'),
        ('egg', 'cosy'),
        ('egg', 'warmer'),
        ('egg', 'spoon'),
        ('egg', 'holder'),
        ('christmas', 'balls'),
        ('candle', 'holder'),
        ('tea', 'towel'),
        ('tea', 'towels'),
        ('fairy', 'lights'),
        ('popcorn', 'holder'),
        ('memo', 'holder'),
        ('soap', 'holder'),
        ('card', 'holder'),
        ('fairy', 'cake'),
        ('fairy', 'cakes'),
        ('box', 'candle'),
        ('boxed', 'candle'),
        ('tutti', 'frutti'),
        ('christmas', 'tree')
    ]
    for combo in word_combinations:
        pattern = r'\b{}\b \b{}\b'.format(combo[0], combo[1])
        replacement = '{}-{}'.format(combo[0], combo[1])
        text = re.sub(pattern, replacement, text)
    return text


# Apply the function to the 'description' column
df['description'] = df['description'].apply(lambda x: hyphenate_words(x))
df

,index,invoice_number,product_id,description,quantity,invoice_date,unit_price,customer_id,country,line_price
0,0,489434,85048,15cm christmas glass ball 20 lights,12,2009-12-01,6.95,13085,united kingdom,83.40
1,1,489434,79323P,pink cherry lights,12,2009-12-01,6.75,13085,united kingdom,81.00
2,2,489434,79323W,white cherry lights,12,2009-12-01,6.75,13085,united kingdom,81.00
3,3,489434,22041,"record frame 7"" single size",48,2009-12-01,2.10,13085,united kingdom,100.80
4,4,489434,21232,strawberry ceramic trinket box,24,2009-12-01,1.25,13085,united kingdom,30.00
...,...,...,...,...,...,...,...,...,...,...
49995,71548,495875,21201,tropical honeycomb paper garland,3,2010-01-27,2.55,16550,united kingdom,7.65
49996,71549,495875,44234,assorted circular mobile,5,2010-01-27,0.21,16550,united kingdom,1.05
49997,71550,495875,47590B,pink happy birthday bunting,2,2010-01-27,5.45,16550,united kingdom,10.90
49998,71551,495875,21745,gaolers keys decorative garden,12,2010-01-27,3.75,16550,united kingdom,45.00


In [138]:
# Get unique values
unique_descriptions = df.description.unique()

# Convert to DataFrame
unique_descriptions_df = pd.DataFrame(unique_descriptions, columns=['unique_descriptions'])

# Export to CSV
unique_descriptions_df.to_csv('unique_descriptions.csv', index=False)


In [139]:


def most_common_words(df, column_name, n=10, ignore_colors=True, ignore_numbers=True, ignore_adjectives=True):
    """
    Returns the most common words in a specified column of a DataFrame, optionally ignoring color words, numbers, and adjectives.

    Parameters:
    df (pandas.DataFrame): The DataFrame.
    column_name (str): The name of the column to analyze.
    n (int): The number of most common words to return.
    ignore_colors (bool): Whether to ignore common color words.
    ignore_numbers (bool): Whether to ignore numbers.
    ignore_adjectives (bool): Whether to ignore adjectives.

    Returns:
    list: A list of tuples with the most common words and their counts.
    """
    # List of common color words to ignore
    color_words = set([
        'black', 'white', 'red', 'blue', 'green', 'yellow', 'brown', 'pink', 'orange', 
        'purple', 'grey', 'gray', 'cyan', 'magenta', 'turquoise', 'beige', 'maroon', 
        'navy', 'teal', 'lavender', 'gold', 'silver', 'bronze', 'coral', 'rose', 'ivory',
        'cream'
    ])
    
    # List of common adjectives to ignore
    adjectives = set([
        'glass', 'of', 'small', 'medium', 'large', 't', 'christmas', 'vintage', 'metal',
        'design', 's', 'hanging', 'in', 'spotty', 'with', 'assorted', 'retro', 'paper',
        'spot', 'ceramic', 'and', 'party', 'set', 'wall', 'mini', 'w', 'easter', 'crystal',
        'heart', 'flock', 'wood', 'zinc', 'paisley', 'english', 'love', 'round', 'baroque',
        'wooden', 'new', 'standing', 'pack', 'on', 'skulls', 'colour', 'diamante', 'skulls',
        'lola', 'charlie', 'tutti-frutti', 'f', 'felt', 'floral', 'woodland', 'painted',
        'botanical', 'spots', 'antique', 'the', 'b'
    ])
    
    # Combine all the text in the column into one large string
    text = ' '.join(df[column_name].dropna().astype(str))
    
    # Remove punctuation and split into words
    words = re.findall(r'\b[\w-]+\b', text.lower())
    
    # Optionally filter out color words
    if ignore_colors:
        words = [word for word in words if word not in color_words]

    # Optionally filter out numbers
    if ignore_numbers:
        words = [word for word in words if not word.isdigit()]

    # Optionally filter out adjectives
    if ignore_adjectives:
        words = [word for word in words if word not in adjectives]

    # Count the frequency of each word
    word_counts = Counter(words)
    
    # Return the n most common words
    return word_counts.most_common(n)

# Example usage:
# Assuming df is your DataFrame and 'product_description' is the column name
common_words = most_common_words(unique_descriptions_df, 'unique_descriptions', n=200)

dict = {item[0]: item[1] for item in common_words}

In [140]:
categories = list(dict.keys())
categories

['candle',
 'flower',
 'bag',
 'box',
 'decoration',
 't-light',
 'mug',
 'holder',
 'candles',
 'mirror',
 'necklace',
 'bracelet',
 'bowl',
 'cake',
 'card',
 'sign',
 'tea',
 'garland',
 'bird',
 'ring',
 'stand',
 'incense',
 'cushion-cover',
 'earrings',
 'garden',
 'hearts',
 'charm',
 'butterfly',
 'christmas-tree',
 'door',
 'fairy',
 'plate',
 'rabbit',
 'clock',
 'star',
 'notebook',
 'key',
 'purse',
 'wrap',
 'bead',
 'hot-water-bottle',
 'candy',
 'art',
 'dish',
 'tree',
 'photo-frame',
 'gift',
 'tube',
 'birthday',
 'letter',
 'table',
 'strawberry',
 'hook',
 'cup',
 'flowers',
 'egg',
 'scented',
 'folkart',
 'fairy-cake',
 't-lights',
 'french',
 'tin',
 'bling',
 'chocolate',
 'gingham',
 'stripe',
 'square',
 'lights',
 'cherry',
 'cutlery',
 'skull',
 'stickers',
 'sweetheart',
 'tray',
 'shape',
 'boxes',
 'pot',
 'light',
 'umbrella',
 'jewelled',
 'photo',
 'toy',
 'mat',
 'coffee',
 'string',
 'enamel',
 'dinner',
 'honeycomb',
 'bunny',
 'cases',
 'union',
 '

In [141]:
#categories = list(dict.keys())
#category_df = pd.DataFrame(categories, columns=['category'])

In [142]:
# Function to determine the category
def find_category(description, categories):
    for category in categories:
        if category in description.lower():
            return category
    return 'other'

# Apply the function to create the new column
df['category'] = df['description'].apply(find_category, categories=categories)

In [143]:
products = df[['product_id', 'description', 'category', 'unit_price']].drop_duplicates(subset=['product_id'], keep='first')
customers = df[['customer_id', 'country']].drop_duplicates()
invoices = df[['invoice_number', 'invoice_date', 'customer_id', 'product_id', 'quantity', 'unit_price', 'line_price', 'country']]

In [144]:
products.unit_price.nunique()

116

In [145]:
print(products.shape)
print(customers.shape)
print(invoices.shape)

(2960, 4)
(1272, 2)
(48454, 8)


In [146]:
products.to_csv('../data/clean/products.csv', sep=',', index=False)
customers.to_csv('../data/clean/customers.csv', sep=',', index=False)
invoices.to_csv('../data/clean/invoices.csv', sep=',', index=False)